In [1]:
from  gensim.similarities.docsim import Similarity
import pandas as pd
import numpy as np
from collections import defaultdict
from gensim import corpora
from tqdm import tqdm

C:\Users\Hiran Dissanayake\.conda\envs\nongpu\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df = pd.read_csv("dataset_cleaned.csv")

In [16]:
df.head()

,DALTIX_ID,text
0,43823f3f6826dcfd1f14b5898d742a1ead54937001f980...,velgenreiniger car bike krcher geschikt remspo...
1,2258f16fa7ff7aab35c4534ac645017637d072f2fee9cc...,matte muurverf originals potloodgrijs levis ge...
2,69ec8d4f776200897422a4b8f93d3416a36781fe110d75...,vlakdraad nieten oorspronkelijk ontworpen blac...
3,f03ef881a1d1934c62b9db6b4403a59ae34de96148f362...,toiletmeubel fabulous 40 meest verkochte badme...
4,aaff0aa6db24814ad25d5cb410ded08361ab32bcb953e1...,pickup pictogram zelfklevend weerbestendig een...


In [3]:
texts = [[word for word in row['text'].split()] for index, row in df.iterrows()]

In [4]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [5]:
texts = [[token for token in text if frequency[token] > 1]for text in texts]

In [6]:
dictionary = corpora.Dictionary(texts)
dictionary.save('daltix.dict')
print(dictionary)

Dictionary(77214 unique tokens: ['0,5', '500.0', 'accessoires', 'af', 'alle']...)


In [7]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [8]:
corpora.MmCorpus.serialize('daltix.mm', corpus)

In [9]:
index = Similarity("daltix.sim.index", corpus, num_features=len(dictionary))

In [10]:
i = 0
results = []
for similarities in tqdm(index):
#     print(similarities.argsort()[-3:][::-1], i)
    arr = similarities.argsort()[-3:][::-1]
    indx = np.argwhere(arr==i)
    arr = np.delete(arr, indx)
#     print(i,arr[0],similarities[arr[0]])
    results.append((i,arr[0],similarities[arr[0]]))
    i += 1

  0%|                                                                                       | 0/101781 [00:00<?, ?it/s]C:\Users\Hiran Dissanayake\.conda\envs\nongpu\lib\site-packages\gensim\similarities\docsim.py:517: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = numpy.hstack(shard_results)
100%|██████████████████████████████████████████████████████████████████████████| 101781/101781 [21:04<00:00, 80.48it/s]


In [19]:
results_ = []
for (i,j,k) in tqdm(results):
#     print(i,j,k)
    results_.append((df.iloc[i]['DALTIX_ID'],df.iloc[j]['DALTIX_ID'],k))

100%|████████████████████████████████████████████████████████████████████████| 101781/101781 [00:25<00:00, 3921.03it/s]


In [21]:
pd.DataFrame(results_, columns=['daltix_id_1', 'daltix_id_2', 'similarity']).to_csv("docsim_df.csv", index=False)